In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col logpop, set log() for population data

In [2]:
d[:logpop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6_1 = "
data{
    int total_tools[10];
    real logpop[10];
    int society[10];
}
parameters{
    real a;
    real bp;
    vector[10] a_society;
    real<lower=0> sigma_society;
}
model{
    vector[10] mu;
    sigma_society ~ cauchy( 0 , 1 );
    a_society ~ normal( 0 , sigma_society );
    bp ~ normal( 0 , 1 );
    a ~ normal( 0 , 10 );
    for ( i in 1:10 ) {
        mu[i] = a + a_society[society[i]] + bp * logpop[i];
        mu[i] = exp(mu[i]);
    }
    total_tools ~ poisson( mu );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6.1",  model=m12_6_1, num_warmup=2000,
num_samples=2000, output_format=:mcmcchain);

Input data for cmdstan

In [4]:
m12_6_1_data = Dict("total_tools" => d[:total_tools], "logpop" => d[:logpop], "society" => d[:society]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_1_data, ProjDir, diagnostics=false, CmdStanDir=CMDSTAN_HOME);

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1.stan' at line 15)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.


Inference for Stan model: m12_6_1_model
4 chains: each with iter=(2000,2000,2000,2000); warmup=(0,0,0,0); thin=(1,1,1,1); 8000 iterations saved.

Warmup took (0.86, 0.91, 1.0, 0.88) seconds, 3.7 seconds total
Sampling took (1.1, 0.96, 1.0, 1.0) seconds, 4.1 seconds total

                    Mean     MCSE   StdDev        5%       50%       95%    N_Eff  N_Eff/s    R_hat
lp__             9.3e+02  7.9e-02  3.1e+00   9.2e+02   9.3e+02   9.3e+02  1.5e+03  3.6e+02  1.0e+00
accept_stat__    

Describe the draws

In [6]:
describe(chn)

Iterations = 1:2000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 2000

Empirical Posterior Estimates:
                   Mean            SD         Naive SE        MCSE         ESS   
         lp__  927.985769000  3.0644731149 0.03426185100 0.07712787633 1578.66102
accept_stat__    0.939241435  0.0846281322 0.00094617128 0.00123435771 2000.00000
   stepsize__    0.053805050  0.0062577417 0.00006996368 0.00070400694   79.00988
  treedepth__    5.817750000  0.4832847329 0.00540328758 0.01555408591  965.42194
 n_leapfrog__   69.320000000 30.8212969234 0.34459257538 0.74441719622 1714.22974
  divergent__    0.000000000  0.0000000000 0.00000000000 0.00000000000        NaN
     energy__ -921.478042500  3.9672362974 0.04435505022 0.09405512784 1779.14433
            a    1.115864400  0.7503400070 0.00838905631 0.01546567871 2000.00000
           bp    0.259562232  0.0811952358 0.00090779033 0.00166221421 2000.00000
  a_society.1   -0.204876675  0.2427568297 0.00271410387 0.00417

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*